In [22]:
import pandas as pd

from datetime import datetime

In [27]:
grades = pd.DataFrame({'2017-02-05': {'Hermione': 5, 'Ron': 2},
                       '2017-02-12': {'Hermione': float('nan'), 'Ron': float('nan')},
                       '2017-02-19': {'Hermione': 7.0, 'Ron': float('nan')},
                       '2017-02-26': {'Hermione': float('nan'), 'Ron': 4.0}})

plan = pd.DataFrame({'activity': {0: 'Test', 1: 'Test', 2: 'Seminar', 3: 'Seminar'},
                     'date': {0: '2017-02-05', 1: '2017-02-12', 2: '2017-02-19', 3: '2017-02-26'}})

excuses = pd.DataFrame({'name': {0: 'Hermione', 1: 'Hermione', 2: 'Ron', 3: 'Harry'},
                        'date': {0: '2017-02-10 2017-02-11', 1: '2017-02-12 2017-02-12', 2: '2017-02-19 2017-02-19', 3: '2017-02-19 2017-02-19'},
                        'reason': {0: 'valid', 1: 'valid', 2: 'invalid', 3: 'invalid'}})

# the sample structure of the result variable
result = pd.Series({'Hermione': 4, 'Ron': 2})

In [28]:
grades = grades.T
plan.set_index('date', inplace=True)
excuses[['start_date', 'end_date']] = excuses['date'].str.split(expand=True)
excuses['start_date'] = pd.to_datetime(excuses['start_date'])
excuses['end_date'] = pd.to_datetime(excuses['end_date'])
excuses.drop('date', axis=1, inplace=True)

In [44]:
plan.to_dict()['activity']

{'2017-02-05': 'Test',
 '2017-02-12': 'Test',
 '2017-02-19': 'Seminar',
 '2017-02-26': 'Seminar'}

In [25]:
def main(grades, excuses, plan):
    # prepare dataframes
    grades = grades.T
    plan.set_index('date', inplace=True) 
    excuses[['start_date', 'end_date']] = excuses['date'].str.split(expand=True)
    excuses['start_date'] = pd.to_datetime(excuses['start_date'])
    excuses['end_date'] = pd.to_datetime(excuses['end_date'])
    excuses.drop('date', axis=1, inplace=True)

    cnt_grades = {}
    names = grades.columns
    for name in names:
        cnt_grades[name] = {'Seminar': {'sum': 0, 'cnt': 0},
                            'Test': {'sum': 0, 'cnt': 0}}
    for date, row in grades.iterrows():
        activity = plan[date]
        for name in names:
            value = row[name]
            if pd.isna(value):
                is_valid = is_valid_excuse(excuses, date, name)
                if is_valid:
                    continue
                else:
                    cnt_grades[activity][name]['cnt'] += 1
            else:
                cnt_grades[activity][name]['cnt'] += 1
                cnt_grades[activity][name]['sum'] += value
    
    result = {}
    for name in names:
        avg_seminars = (cnt_grades['Seminar'][name]['sum'] /
                        cnt_grades['Seminar'][name]['cnt'])
        avg_tests = (cnt_grades['Test'][name]['sum'] /
                        cnt_grades['Test'][name]['cnt'])
        result[name] = 0.7*avg_seminars + 0.3*avg_tests
    return result
        

def is_valid_excuse(excuses, date, name):
    date = datetime.strptime(date, "%Y-%m-%d")
    filtered_df = excuses[(excuses['start_date'] <= date) & (excuses['end_date'] >= date)]
    filtered_df = filtered_df[filtered_df['name'] == name]
    if len(filtered_df) == 0:
        return False
    else:
        reason = filtered_df.iloc[0]['reason']
        if reason == 'valid':
            return True
        return False
    
    
main(grades, excuses, plan)

KeyError: '2017-02-05'

In [4]:
excuses

,name,date,reason
0,Hermione,2017-02-10 2017-02-11,valid
1,Hermione,2017-02-12 2017-02-12,valid
2,Ron,2017-02-19 2017-02-19,invalid
3,Harry,2017-02-19 2017-02-19,invalid
